# Lending Club Case Study
### Group - Avinash Kumar, Krishna R Mojamdar
### Version 1 - 09/11/2021 Kaggle

## About Case Study

The aim of this case study is to get an idea of how real world business problems are solved using EDA. 

> * Applying different EDA techniques 
> * Develop an understanding of risk analytics in the BFSI domain
> * How the data is used to minimise loss of money while leanding to customers.

## Business Understanding 

When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile.

**There are two types of risks associated with the bank’s decision:**

> * If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company
> * If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company

The data given contains the information about past loan applicants and whether they ‘defaulted’ or not. The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.

When a person applies for a loan, there are two types of decisions that could be taken by the company:

> **Loan accepted:** If the company approves the loan, there are 3 possible scenarios described below:
> * **Fully paid**: Applicant has fully paid the loan (the principal and the interest rate)
> * **Current**: Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. These candidates are not labelled as 'defaulted'.
> * **Charged-off**: Applicant has not paid the instalments in due time for a long period of time, i.e. he/she has defaulted on the loan 
>
> **Loan rejected:** Not Considered as there is no transactional history of those applicants with the company and so this data is not available with the company



# Data Import and Cleanup

 **Importing Packages**

In [ ]:
# Importing Packages

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 1000)

**Importing Dataset**


In [ ]:
# Data Import
df = pd.read_csv("../input/lending-club-data/loan.csv", index_col=False)

# Display Top 10 Columns
df.head(10)

**Dataset Understanding & Cleanup**



In [ ]:
#Shape of Data 
df.shape

There are 39717 Rows and 111 Columns

Now lets check for columns with null values

In [ ]:
#Getting count of null values in each column
df.isnull().sum()

It appears that there are many columns with null values

Lets get the exact count


In [ ]:
print("Null values")
print(len(df.columns[df.isna().any()]))

print("No Null values")
print(len(df.columns[df.notna().all()]))

There are 68 Columns with null values

Lets identify the percentage of NA Values for further consideration

In [ ]:
# Checking Percentage of Null values in each column
percentage = round(df.isnull().sum() * 100 /len(df.index))
percentage

There are a lot of columns with more than 30% null values. 

Lets remove columns having more than 30% null values

In [ ]:
# Removing all columns with more than 30% NULL values
threshold = len(df.index) * 0.3
df2 = df.dropna(thresh = threshold,axis = 1)

# Dropping columns of no relevance as of the moment
df2 = df2.drop(['id', 'desc', 'url', 'mths_since_last_delinq'], axis=1)

# Calculate Missing Percentage
percentage = df2.isnull().sum() * 100 /len(df)

# Verifying Drop
percentage

# Check DF shape
df2.shape

There are now 51 Columns

Now lets further check for missing values 

In [ ]:
# Further Missing Values - Chekcing Percentage

missing = round(100*(df2.isnull().sum()/len(df2)))
missing[missing != 0]

emp_title has 6% missing values whereas emp_length has 3% missing values. These columns contains information about customers and is a feature. The NA values can cause incorrect calulations and thus needs to be imputed.

For pub_rec_bankruptcies, the NA value can be replaced with 0

In [ ]:
# Imputing emp_title mode for the missing values in the column  from df2
df2.emp_title.fillna(df2.emp_title.mode()[0],inplace = True)

#Imputing emp_length mode for the missing values in the column from df2
df2.emp_length.fillna(df2.emp_length.mode()[0],inplace = True)

#replacing NA with 0 in pub_rec_bankruptcies

df2['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0)

# Check DF

missing = round(100*(df2.isnull().sum()/len(df2)))
missing[missing != 0]


Identify Categorical columns and check for features

In [ ]:
# Count the number of unique values in such columns and drop columns with no useful data
df2.nunique().sort_values()

Columns pymnt_plan, initial_list_status,             acc_now_delinq, application_type, policy_code,               collections_12_mths_ex_med, tax_liens,               chargeoff_within_12_mths have common or no usefull value and hence can be dropped.

In [ ]:
# Dropping all the columns with only one unique value
Loan_DF = df2.drop(['pymnt_plan',
                'delinq_amnt',
                'initial_list_status',
                'acc_now_delinq',
                'application_type',
                'policy_code',
                'collections_12_mths_ex_med',
                'tax_liens',
                'chargeoff_within_12_mths',
                ] , axis=1)


Now it is noted that column term, int_rate and emp_length can be derived into numerical values.

In [ ]:
# Removing string months from column term
Loan_DF['term'] = Loan_DF['term'].str.rstrip('months')
# Removing string % from column rate
Loan_DF['int_rate'] = Loan_DF['int_rate'].str.rstrip('%')
# Removing string years from column emp_length
Loan_DF['emp_length'] = Loan_DF['emp_length'].str.rstrip('years ')
# Removing prefix < from column emp_length - Considering less than 1 year as 1 year
Loan_DF['emp_length'] = Loan_DF['emp_length'].str.lstrip('< ')
# Removing prefix + from column emp_length
Loan_DF['emp_length'] = Loan_DF['emp_length'].str.rstrip('+ ')


Now that we have the dataset ready, we will look at the different datatypes

In [ ]:
Loan_DF.dtypes

It appears that the columns are objects and needs to be converted to appropriate data types


In [ ]:
# Converting required columns to numeric
numeric_type = ['loan_amnt', 'int_rate', 'funded_amnt', 'funded_amnt_inv',
                   'installment', 'emp_length', 'annual_inc', 'dti', 'revol_bal',
                   'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
                   'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
                   'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt',
                   'total_acc', 'term']
Loan_DF[numeric_type] = Loan_DF[numeric_type].apply(pd.to_numeric)

# Converting Required columns to Category

category_type = ['grade', 'sub_grade', 'home_ownership', 'verification_status',
                 'loan_status', 'purpose', 'addr_state', 'delinq_2yrs', 'inq_last_6mths',
                 'open_acc', 'pub_rec', 'pub_rec_bankruptcies']
Loan_DF[category_type] = Loan_DF[category_type].apply(lambda x: x.astype('category'))

Loan_DF.dtypes

As per business understanding, customers with Status as "Current" are not labelled as 'defaulted'.

In [ ]:
# Check shape of DF
Loan_DF.shape

In [ ]:
# Preparing DF for EDA analysis, considering loan status as Fully Paid, Charged off and Current
loan = Loan_DF[Loan_DF['loan_status'].isin(['Fully Paid', 'Charged Off','Current'])]
loan['loan_status'].value_counts()

# **Exploratory Data Analysis**

## **Univariate Analysis**

Now that the dataset has been prepared, we will now go ahead with Univariate analysis.

We would first look at one of the key columns loan_amnt to check the distribution for different loan status

In [ ]:
# Supressing Warnings
import warnings
warnings.filterwarnings("ignore")

#Plotting
sns.boxplot(y = loan['loan_amnt'],x = loan['loan_status'])
plt.show()

From the above it is clear that the loan amount ranges from 0 to 35000 with the mean being an amount of 10000.

Lets now try to find the outliers. We would want to get rid of the outliers to get an even distribution.

In [ ]:
loan.describe()

In [ ]:
#Removing Outliers - Caluclating outliers and removing from DF.

Q1,Q2,Q3 = loan['loan_amnt'].quantile([0.25,0.5,0.75])
IQR = Q3-Q1
high_outlier = Q3+1.5*IQR
print("The data points above ",high_outlier, "will be considered as outliers")
loan = loan.loc[loan['loan_amnt'] < high_outlier]


We will have a look at the status of the loans and try to infer the meaning out of it. 

In [ ]:
# Plotting Status of Loans
sns.countplot(x=loan['loan_status'], data=loan[-(loan.loan_status == '0')])
plt.show()

Is is clear from the graph that the majority of the loans have been fully paid. 

Now lets have a look at the Grade and Sub Grade


In [ ]:
# Plotting Grade by Mean to establish grade category
loan.groupby(['grade'],as_index=False).agg(total_loan = ('loan_amnt','sum'),mean_loan = ('loan_amnt','mean')).plot.line(x = 'grade',y = 'mean_loan')

Grade A is basic loan and Grade G is premium loan 

In [ ]:
#Plotting Grade 

sns.countplot(x=loan['grade'])
plt.show()

In [ ]:
# Plotting Sub Grade
plt.figure(figsize = (20,8))
sns.countplot(x=loan['sub_grade'])
plt.show()

Most of the loans fall under grade A, B and Sub Grade A4 to B5. Hence most of the loans are low graded loans.

Note: Its assumed that Grade G is the highest based on the mean.

Lets now know more about our applicants

To begin with, we will look at categorical columns

In [ ]:
# Plotting all categorical colums with a loop
for columns in loan.select_dtypes('category'):
    plt.figure(figsize = (20,5))
    sns.countplot(x=loan[columns])
    plt.show()

From the above, it can be seen that 

* Grade B has the highest number of applicants
* A high number of the applicants live in rented and mortgaged houses.
* A very few (less than 2500) own a house.
* A large number of customers were not verified
* Majority of the loans are fully paid.
* Debt consolidation seems to be the major purpose of taking a loan.
* California (CA) has the highest number of applicants.
* More than 20000 of the applicants had Credit inquires over the last 6 months.
* A large population has more than 3 open credit lines

Now lets look at the annual income

In [ ]:
# Derive Category for Income Group
loan['income_group'] = pd.cut(loan['annual_inc'], [x for x in range(0, 100000, 10000)])
# Range End at 100000 for visuals, there is data beyond 100000.

# Plotting Income Groups
plt.figure(figsize = (20,8))
sns.countplot(x=loan['income_group'])
plt.show()

Income group 40000-60000 has the highest number of applicants in the dataset.


Now lets check the loan amount that were taken


In [ ]:
# Segmenting Loans to category
loan['loan_amt_category'] = loan['loan_amnt'].apply(lambda x: 'Low' if x<5000 else ( 'Medium' if (x>=5000 and x<15000) else 'High'))

loan['loan_amt_category'].value_counts()

# Plot Loans as per category
plt.figure(figsize = (15,8))
sns.barplot(x=loan['loan_amt_category'], y=loan['loan_amnt'], data=loan)
plt.show()

Majority of the loans are in the High Category.


## **Segmented Univariate Analysis**

In [ ]:
# Checking Loan status as of Loan amount category
sns.countplot(x=loan['loan_amt_category'], hue=loan['loan_status'])
plt.show()

Its noted that there is a high Charge off for loans falling under Medium Loan Category

Lets see if home ownership affects the loan status.

In [ ]:
sns.countplot(x=loan['home_ownership'], hue=loan['loan_status'])
plt.show()

It can be seen that applicants on Mortgage and Rent have the highest Charge off.

Lets check if purpose has any relation

In [ ]:
plt.figure(figsize = (25,6))
sns.countplot(x=loan['purpose'], hue=loan['loan_status'])
plt.show()

It can be observed that even though debt consolidation has the highest fully paid applicants, it also has the highest charge offs. 

## **Bivariate Analysis**

Lets look at the correlation of the columns


In [ ]:
# plotting correlation heatmap
plt.figure(figsize = (20,8))
sns.heatmap(loan.corr(), cmap="Greens", annot=True)
plt.show()

Creating a dataset for charged off loans

In [ ]:
loan  = loan[loan['loan_status'] == 'Charged Off']

In [ ]:
# Plotting Purpose and Home Ownership
loan_home_pupose = loan.groupby(['purpose','home_ownership']).agg(total_loan = ('loan_amnt','sum'))
loan_home_pupose.reset_index(inplace = True)
plt.figure(figsize = (20,4))
sns.barplot(x = loan_home_pupose['purpose'],y = loan_home_pupose['total_loan'],hue=loan_home_pupose['home_ownership'],)

Size = Sum of total Loan charged off

Style = Purpose of the Loan

As seen from the plot below a huge spike is observed in the total loans that have been waivered is for people who have who do not own a house and borrowed money for debt consolidation

In [ ]:
df_inc = loan.groupby(['purpose','home_ownership','verification_status']).agg(sum_loan = ('loan_amnt','sum'), )
df_inc.reset_index(inplace = True)
plt.figure(figsize = (120,100))
g = sns.relplot(x = df_inc['verification_status'],y = df_inc['home_ownership'],style = df_inc['purpose'],hue  = df_inc['purpose'],sizes = (10,700),size = df_inc['sum_loan'],legend = 'auto',palette = 'hls')
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))

From the above graph it can be inferred that hight amount of loan is being charged off from customers who are verified and have either mortgaged their home or customers who stay in rented homes. The same is true for customers who take loans for debt consolidation.

Note: The size of the points on the plot are proportional to the total sum of the loan and relative to the purpose for which the loan was taken. Each shape and color represent various purposes as can be seen in the legend.

In [ ]:
# Plotting Grade vs Verification Status
df_grade = loan.groupby(['grade','verification_status'],as_index=False).agg(total_loan = ('loan_amnt','sum'),mean_loan = ('loan_amnt','mean'),member_id = ('member_id','count'))
plt.figure(figsize = (20,8))
sns.lineplot(x = 'grade',y= 'total_loan',hue = 'verification_status',data = df_grade)
sns.lineplot(x = 'grade',y= 'member_id',hue = 'verification_status',data = df_grade,linestyle='dashed',ax = plt.twinx(),legend = False)

B grade loans tend to be charged off for verified sources

In [ ]:
# Checking zip codes for the total loan amount
df_zip = loan.groupby(['zip_code']).agg(total_loan_amnt = ('loan_amnt','sum'))
df_zip.sort_values(by = 'total_loan_amnt',ascending=False).plot.line()

A huge spike is observed for people who reside in the zip code "945xx"

In [ ]:
# Evaluating Grade and amount for customers in zip code 994xx.

df_char_off_945xx = loan.loc[loan['zip_code'] == '945xx']
df_grade_zip = df_char_off_945xx.groupby(['grade','verification_status']).agg(total_loan = ('loan_amnt','sum'),mean_loan = ('loan_amnt','mean'),ids = ('member_id','count'))
df_grade_zip.reset_index(inplace = True)
plt.figure(figsize = (20,8))
sns.lineplot(x = 'grade',y= 'total_loan',hue = 'verification_status',data = df_grade_zip,alpha = 0.5)
sns.lineplot(x = 'grade',y= 'ids',hue = 'verification_status',data = df_grade_zip,linestyle='dashed',ax = plt.twinx(),legend = True)

Contrary to what was observed in the overall trend, the most charged off loans for zip code 945xx was for grade C and grade E. Grade G loans were not granted to people residing in this zip code


# **Summary**

**The following actions were performed for preparing the dataset for EDA.**

* Identify the shape of data frame
* Identify the colums having null values
* Evaluate the percentage of null values
* Dropping columns having more than 30% null values
* Imputing the columns which has minimal amout of null values
* Identify column having numerical and categorical data
* Identify data types of columns
* Converting data types of columns based on the type of data as required


**Data Analysis**

From the Lending Club Dataset

The loan amount ranges from 0 to 35000 with the mean being an amount of 10000. The dataset had a few outliers and any amount above 29250 was considered to be the outlier and removed from the dataset under consideration to get an even distribution.
It was observed that a majority of the loans have been fully paid and a substantial number were charged off.
The dataset offered grades (A to G) for each loan with G being the premium loans (High loan amount).
Given the fact, it was observed that most of the loans fall under grade A, B and Sub Grade A4 to B5. Hence most of the loans are low graded loans i.e Low loan amount. 
A quick analysis of the identified categorical data suggested the following:

* Grade B has the highest number of applicants
* A high number of the applicants live in rented and mortgaged houses. A few own a house.
* Majority of the loans are fully paid.
* Debt consolidation seems to be the major purpose of taking a loan.
* More than 20000 of the applicants had credit inquires over the last 6 months.
* A large population has more than 3 open credit lines
* Income group 40000-60000 has the highest number of applicants in the dataset.
* Majority of the loans are in the High category.
* It was noted that there is a high Charge off for loans falling under Medium Loan Category
* It can be seen that applicants on Mortgage and Rent have the highest Charge off.
* B grade loans tend to be charged off for verified sources
* California (CA) has the highest number of applicants.
* A huge spike is observed for people who reside in the zip code "945xx"

A huge spike was observed in the total loans that have been waivered is for people who have who do not own a house and borrowed money for debt consolidation and a high amount of loan is being charged off from customers who are verified and have either mortgaged their home or customers who stay in rented homes. The same is true for customers who take loans for debt consolidation.
Contrary to what was observed in the overall trend, the most charged off loans for zip code 945xx was for grade C and grade E. Grade G loans were not granted to people residing in this zip code
